In [1]:
from tulip.core.collection import TulipCollection
from tulip.data.haver import HaverClient as hc
from IPython.display import Markdown
import pandas as pd
from tulip.plots import plot_line, plot_lines, switch_trace_to_secondary_axis

Haver path setting remains unchanged.



## New Zealand

### (a) Real GDP Growth, YoY

In [2]:
hc.get_series("yryr%(NZSGDPC@ANZ)").plot()

### (b) Nominal GDP Growth, YoY

In [3]:
hc.get_series("yryr%(NZSGDP@ANZ)").plot()

### (c) GDP Growth in Dashboard

In [4]:
growth_stats = {
    "Real GDP": "yryr%(NZSGDPC@ANZ)",
    "Gross Fixed Capital Formation": "yryr%(NZSNFC@ANZ)",  # Equivalent to fixed investment
    "Change in Inventories": "NZSNBC@ANZ",  # This can't be expressed in %
    "Government Consumption": "yryr%(NZSNGC@ANZ)",
    "Private Consumption": "yryr%(NZSNCC@ANZ)",
    "Exports": "yryr%(NZSNXC@ANZ)",
    "Imports": "yryr%(NZSNMC@ANZ)",
}

growth_stats_collection = []
for k, v in growth_stats.items():
    growth_stats_collection.append(hc.get_series(v))

growth_stats_collection = TulipCollection(growth_stats_collection)

growth_stats_collection.good_is = {
    "yryr%(NZSGDPC@ANZ)": 1,
    "yryr%(NZSNFC@ANZ)": 1,
    "NZSNBC@ANZ": 1,
    "yryr%(NZSNGC@ANZ)": 1,
    "yryr%(NZSNCC@ANZ)": 1,
    "yryr%(NZSNXC@ANZ)": 1,
    "yryr%(NZSNMC@ANZ)": -1,
}
growth_stats_collection.dashboard.table()

,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
"New Zealand: Gross Domestic Product (SA, Mil.Chn.Q2:09-Q1:10.NZ$)",0.03,2025-06-30,0.13,-0.10,-0.03,1.5,0.52,2025-11-12 19:25:00
"New Zealand: GDP: Gross Fixed Capital Formation (SA, Mil.Chn.Q2:09-Q1:10.NZ$)",-3.2,2025-06-30,-3.4,0.19,0.03,2.7,2.2,2025-11-12 19:25:00
"New Zealand: GDP: Change in Stocks (SA, Mil.Chn.Q2:09-Q1:10.NZ$)",-136.0,2025-06-30,-495.0,359.0,0.79,402.0,144.0,2025-11-12 19:25:00
"New Zealand: GDP: Government Consumption Expenditure(SA,Mil.Chn.Q2:09-Q1:10.NZ$)",1.6,2025-06-30,1.8,-0.20,-0.10,3.1,1.3,2025-11-12 19:25:00
"New Zealand: GDP: Private Consumption Expenditure (SA, Mil.Chn.Q2:09-Q1:10.NZ$)",1.5,2025-06-30,0.96,0.57,0.18,1.4,2.5,2025-11-12 19:25:00
"New Zealand: GDP: Exports of Goods and Services (SA, Mil.Chn.Q2:09-Q1:10.NZ$)",0.72,2025-06-30,3.6,-2.9,-0.53,-2.5,-1.3,2025-11-12 19:25:00
"New Zealand: GDP: Imports of Goods and Services (SA, Mil.Chn.Q2:09-Q1:10.NZ$)",2.2,2025-06-30,0.27,1.9,0.30,2.8,1.5,2025-11-12 19:25:00


In [5]:
growth_stats_collection.dashboard.plots(show_0=True)

## #2 Trade

In [6]:
df_tb = hc.get_series("NZSBCGS@ANZ").ts.rename("Trade Balance (NZD mn)")
df_er = hc.get_series("NZNXUSL@ANZ").ts.rename("NZD/USD")

fig = plot_line(
    blue=df_tb,
    red=df_er,
    title="<b>New Zealand: Trade Balance vs NZD/USD Exchange Rate</b>",
    tick_suffix="",
    years_limit=10,
    watermark=False,
    source="Haver",
    # figsize=(1000, 600)
)

fig.data[1].update(yaxis="y2")

fig.update_layout(
    yaxis=dict(
        title=dict(text="Trade Balance (NZD mn)", font=dict(color="blue")),
        tickfont=dict(color="blue"),
    ),
    yaxis2=dict(
        title=dict(text="NZD/USD", font=dict(color="red")),
        tickfont=dict(color="red"),
        overlaying="y",
        side="right",
    ),
)

fig.show()

## #3 PMI

In [7]:
fig = hc.get_series("NZSPCIGC@ANZ").plot()
fig.add_hline(y=50, line_width=1, line_dash="dash", line_color="black")
fig.show()

 ## #4 Confidence Indicators

In [8]:
fig = hc.get_series("NZSRMIDX@ANZ").plot()
fig.add_hline(y=100, line_width=1, line_dash="dash", line_color="black")
fig.show()

## #5 Inflation

### (a) CPI

In [9]:
hc.get_series("yryr%(NZSPC@ANZ)").plot()

### (b) Inflation Expectations

In [10]:
df_1year = hc.get_series("N196VEC1@INTSRVYS").ts.rename("1 Year CPI Expectations")
df_2year = hc.get_series("N196VEC2@INTSRVYS").ts.rename("2 Year CPI Expectations")
df_5year = hc.get_series("N196VEC5@INTSRVYS").ts.rename("5 Year CPI Expectations")
df_10year = hc.get_series("N196VECX@INTSRVYS").ts.rename("10 Year CPI Expectations")

df = pd.concat([df_1year, df_2year, df_5year, df_10year], axis=1).dropna()
df = df.dropna()

fig = plot_lines(
    df[
        [
            "1 Year CPI Expectations",
            "2 Year CPI Expectations",
            "5 Year CPI Expectations",
            "10 Year CPI Expectations",
        ]
    ],
    title="<b>New Zealand:CPI Expectations 1, 2, 5, and 10 Years From Now Policy Rate",
    tick_suffix="%",
    watermark=False,
    years_limit=10,
    source="Haver",
    figsize=(1000, 600),
)

fig.show()


### (c) CPI Weights

In [11]:
df_food = hc.get_series("NZNCWF@ANZ").ts.rename("Food")
df_alcohol = hc.get_series("NZNCWB@ANZ").ts.rename("Alcohol and Tobacco")
df_clothing = hc.get_series("NZNCWA@ANZ").ts.rename("Clothing")
df_housing = hc.get_series("NZNCWH@ANZ").ts.rename("Housing")
df_furnishing = hc.get_series("NZNCWG@ANZ").ts.rename("Furnishing")
df_health = hc.get_series("NZNCWM@ANZ").ts.rename("Health")
df_transport = hc.get_series("NZNCWT@ANZ").ts.rename("Transport")
df_communication = hc.get_series("NZNCWC@ANZ").ts.rename("Communication")
df_recreation = hc.get_series("NZNCWR@ANZ").ts.rename("Recreation")
df_education = hc.get_series("NZNCWE@ANZ").ts.rename("Education")
df_miscellaneous = hc.get_series("NZNCWO@ANZ").ts.rename("Miscellaneous")

df = pd.concat(
    [
        df_food,
        df_alcohol,
        df_clothing,
        df_housing,
        df_furnishing,
        df_health,
        df_transport,
        df_communication,
        df_recreation,
        df_education,
        df_miscellaneous,
    ],
    axis=1,
).dropna()
df = df.dropna()

fig = plot_lines(
    df[
        [
            "Food",
            "Alcohol and Tobacco",
            "Clothing",
            "Housing",
            "Furnishing",
            "Health",
            "Transport",
            "Communication",
            "Recreation",
            "Education",
            "Miscellaneous",
        ]
    ],
    title="<b>New Zealand CPI Weightings",
    tick_suffix="%",
    watermark=False,
    years_limit=10,
    source="Haver",
    figsize=(1000, 600),
)

fig.show()

### (d) Inflation Nowcast

In [12]:
# #Gonna be doing a regression model using price indexes that are all highly correlated with inflation
# Food price index - NZSPF@ANZ
# NZSPF@ANZ - food
# NZNPEHE@ANZ - household electricity
# NZNPEHF@ANZ - household firewood
# NZNPEHG@ANZ - household gas
# NZNPEHP@ANZ - household petrol
# NZNPEHM@ANZ - household alternative motor fuels
# NZNPECE@ANZ - commercial electricity
# NZNPECL@ANZ - commercial LPG
# NZNPECG@ANZ - commercial natural gas
# NZNPECP@ANZ - commercial bulk control
# NZNPECD@ANZ - commercial bulk diesel
# NZNPON@ANZ - commodity price index

# Flow (rent): NZNHJRT@ANZ
# Stock (actual property): NZNHJRK@ANZ
# Detached housing: NZNPHD@ANZ
# NOWCAST

## #6 Unemployment

### (a) Unemployment Rate

In [13]:
fig = hc.get_series("NZSELUR@ANZ").plot()
fig.add_hline(y=100, line_width=1, line_dash="dash", line_color="black")
fig.show()

### (b) NAIRU

In [14]:
beta = 0.5

urate_exp_1y = hc.get_series("N196VEU1@INTSRVYS").ts.rename("Unemployment Exp 1Y")
urate_exp_2y = hc.get_series("N196VEU2@INTSRVYS").ts.rename("Unemployment Exp 2Y")
infl_exp_1y = hc.get_series("N196VEC1@INTSRVYS").ts.rename("Inflation Exp 1Y")
infl_exp_2y = hc.get_series("N196VEC2@INTSRVYS").ts.rename("Inflation Exp 2Y")

df_exp = pd.concat(
    [urate_exp_1y, urate_exp_2y, infl_exp_1y, infl_exp_2y], axis=1
).dropna()
df_exp["Inflation Delta"] = df_exp["Inflation Exp 2Y"] - df_exp["Inflation Exp 1Y"]
df_exp["NAIRU Estimate (from 1Y UR)"] = (
    df_exp["Unemployment Exp 1Y"] + df_exp["Inflation Delta"] / beta
)


nairu_est = df_exp["NAIRU Estimate (from 1Y UR)"].rename("Estimated NAIRU")
urate_exp_1y = df_exp["Unemployment Exp 1Y"].rename("Expected Unemployment (1Y)")

fig = plot_lines(
    [nairu_est, urate_exp_1y],
    title="Market-Implied NAIRU vs. Expected Unemployment",
    tick_suffix="%",
    y_axis_label="Unemployment Rate (%)",
    source="INTSRVYS / Haver",
    years_limit=10,  # or however much range you want to show
)

fig.show()

## #7 Fiscal Balance

### (a) Fiscal Dashboard

In [15]:
fiscal_stats = {
    "Operating Balance": "NZNFGOB@ANZ",
    "Revenue: Sovereign Power": "NZNFGRS@ANZ",
    "Revenue: Operations": "NZNFGRC@ANZ",
    "Social Security": "NZNFESI@ANZ",
    "Education": "NZNFEED@ANZ",
    "Health": "NZNFEHC@ANZ",
    "Housing": "NZNFEND@ANZ",
    "Transport & Communication": "NZNFEAT@ANZ",
}

fiscal_collection = [hc.get_series(code) for code in fiscal_stats.values()]
fiscal_dashboard = TulipCollection(fiscal_collection)

fiscal_dashboard.good_is = {
    "NZNFGOB@ANZ": 1,
    "NZNFGRS@ANZ": 1,
    "NZNFGRC@ANZ": 1,
    "NZNFESI@ANZ": -1,  # assuming higher gov expense = worse
    "NZNFEED@ANZ": -1,
    "NZNFEHC@ANZ": -1,
    "NZNFEND@ANZ": -1,
    "NZNFEAT@ANZ": -1,
}

fiscal_dashboard.dashboard.table()

,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
New Zealand: Govt Finance: Total Crown Operating Balance (Mil.NZ$),"-4,400",2025-12-31,"-8,365","3,965",0.31,"3,965","3,965",2025-11-12 19:26:00
NZ: Govt Finance: Revenue Levied by the Crown's Sovereign Power (Mil.NZ$),"130,826",2025-12-31,"129,326","1,500",0.41,"1,500","1,500",2025-11-12 19:26:00
New Zealand: Govt Finance: Revenue Earned thr the Crown's Operations (Mil.NZ$),"38,985",2025-12-31,"38,021",964.0,0.35,964.0,964.0,2025-11-12 19:26:00
New Zealand: Govt Finance: Expenses: Social Security and Welfare (Mil.NZ$),"57,635",2025-12-31,"53,996","3,639",1.1,"3,639","3,639",2025-11-12 19:26:00
New Zealand: Govt Finance: Expenses: Education (Mil.NZ$),"22,308",2025-12-31,"21,186","1,122",1.6,"1,122","1,122",2025-11-12 19:26:00
New Zealand: Govt Finance: Expenses: Health (Mil.NZ$),"29,853",2025-12-31,"29,840",13.0,0.01,13.0,13.0,2025-11-12 19:26:00
New Zealand: Govt Finance: Expenses: Defense (Mil.NZ$),"3,226",2025-12-31,"3,125",101.0,1.1,101.0,101.0,2025-11-12 19:26:00
New Zealand: Govt Finance: Expenses: Transport and Communication (Mil.NZ$),"15,830",2025-12-31,"16,733",-903.0,-0.57,-903.0,-903.0,2025-11-12 19:26:00


In [16]:
fiscal_dashboard.dashboard.plots(show_0=True)

### (b) Fiscal Indicators

In [17]:
fiscal_df = pd.concat([s.ts.rename(s.name) for s in fiscal_collection], axis=1).dropna()
fiscal_indexed = fiscal_df.divide(fiscal_df.iloc[0]).multiply(100)
fig = plot_lines(
    fiscal_indexed,
    title="<b>New Zealand Fiscal Indicators (Indexed to 100)</b>",
    axis_title="Index (Base = 100)",
    years_limit=10,
    source="Haver",
    watermark=False,
)

fig.add_hline(y=100, line_width=1, line_dash="dash", line_color="black")
fig.show()

### (c) Operating Balance

In [18]:
hc.get_series("NZNFGOBP@ANZ").plot()

## #8 Credit Creation

### (a) Debt Composition

In [19]:
# df_dd = hc.get_series("SESZLDDG@NORDIC").ts.rename("Domestic Debt")
# df_hd = hc.get_series("NZSZDHP@ANZ").ts.rename("Household Debt")
# df_nfd = hc.get_series("SESZDNP@NORDIC").ts.rename("Nonfinancial Corporations Debt")
# df_fd = hc.get_series("SESZDFP@NORDIC").ts.rename("Financial Corporations Debt")
# df_gd = hc.get_series("SESZDGP@NORDIC").ts.rename("Government Debt")
# df_nd = hc.get_series("SESZLSP@NORDIC").ts.rename("Nonprofit Institutions Serving Households' Debt")
#
# combined_df = pd.concat([df_dd, df_hd, df_nfd, df_fd, df_gd, df_nd], axis=1).dropna()
#
# fig = plot_lines(
#     combined_df,
#     title="",
#     tick_suffix="%",
#     y_axis_label="Debt Outstanding as a % of SA GDP</b>",
#     x_axis_label="Date",
#     watermark=False,
#     years_limit=10,
#     source="Haver",
#     logo=False
#     # figsize=(1000, 600)
# )
# fig.show()

### (b) Policy Rates

In [20]:
hc.get_series("NZNROCR@ANZ").plot()

### (c) Real Interest Rates

In [21]:
df_policy = hc.get_series("NZNROCR@ANZ").ts.rename("Policy Rate")
df_cpi = hc.get_series("yryr%(NZSPC@ANZ)").ts.rename("CPI Inflation")

df = pd.concat([df_policy, df_cpi], axis=1).dropna()
df["Real Rate (Policy - CPI)"] = df["Policy Rate"] - df["CPI Inflation"]
df = df.dropna()

fig = plot_lines(
    df[["Policy Rate", "CPI Inflation", "Real Rate (Policy - CPI)"]],
    title="<b>New Zealand: Policy Rate, Inflation, and Real Interest Rate</b>",
    tick_suffix="%",
    watermark=False,
    years_limit=10,
    source="Haver",
    figsize=(1000, 600),
)

fig.show()

## #9 Assets

### (a) Real Estate

In [22]:
df_policy = hc.get_series("NZNROCR@ANZ").ts.rename("Policy Rate")
df_2m = hc.get_series("NZNRSM2Y@ANZ").ts.rename(
    "2Y Mortgage Yield"
)  # Not seasonally adjusted
df_5m = hc.get_series("NZNRSM5Y@ANZ").ts.rename("5Y Mortgage Yield")
df_hp = hc.get_series("NZNPHD@ANZ").ts.rename(
    "Housing Prices"
)  # Detached housing makes up the majority of housing in New Zealand

df_all = pd.concat([df_2m, df_5m, df_policy, df_hp], axis=1).dropna()

fig = plot_lines(
    df_all,
    title="<b>New Zealand Mortgage Yields, Policy Rate, and Housing Prices</b>",
    width=1000,
    height=600,
    tick_suffix="%",
    watermark=False,
    source="Haver",
    default_y_range=(-10, 15),
)

switch_trace_to_secondary_axis(
    fig,
    trace_names="Housing Prices",
    secondary_axis_title="Housing Price Index (2003 Q4-03 = 1000)",
    tick_suffix="",  # No % on right
)

fig.update_layout(yaxis=dict(title="Mortgage Yields & Policy Rate (%)"))

fig.show()

### (b) Stock Market

In [23]:
nzx = hc.get_series("S196K50@INTWKLY").ts.rename("NZX50")
spx = hc.get_series("SP5TRE@USECON").ts.rename("S&P500")


df = pd.concat([nzx, spx], axis=1).dropna()
df = df[df.index >= "2020-01-01"]

df_rebased = df / df.iloc[0] * 100

fig = plot_lines(
    df_rebased,
    title="<b>Rebased Index Return Comparison: NZX50 vs. S&P 500 (Start = 100)</b>",
    axis_title="Index Level (Rebased)",
    source="Haver",
    watermark=False,
    width=1000,
    height=600,
)

fig.show()

## #10 RBNZ Sentiment

In [24]:
rbnz_sentiment = {
    "Business Expectations": "NZNVOB@ANZ",
    "Consumer Confidence": "NZSRMIDX@ANZ",
    "General Business Situation": "NZSVBOSV@ANZ",
    "1Y CPI Expectations": "NZNVEC1Y@ANZ",
    "2Y CPI Expectations": "NZNVEC2Y@ANZ",
    "5Y CPI Expectations": "NZNVEC5Y@ANZ",
}

rbnz_sentiment_collection = []
for k, v in rbnz_sentiment.items():
    rbnz_sentiment_collection.append(hc.get_series(v))

rbnz_sentiment_collection = TulipCollection(rbnz_sentiment_collection)
rbnz_sentiment_collection.dashboard.table()

,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
"New Zealand: NBNZ Business Outlook Survey: Business Confidence (NSA, % Balance)",58.1,2025-10-31,49.6,8.5,0.67,8.8,-7.6,2025-11-12 19:26:00
"New Zealand: Roy Morgan Survey: Overall Index (SA, Index)",92.0,2025-10-31,92.9,-0.90,-0.21,-8.7,1.1,2025-11-12 19:26:00
"NZ: NZIER Bus Op Surv: Exp General Bus Situation, Econ-Wide: Next 6 Mos (SA, %)",15.3,2025-09-30,25.8,-10.4,-0.50,5.5,18.9,2025-11-12 19:26:00
"New Zealand: Expected Annual CPI: 1 Year from Now (NSA, % Chg)",2.4,2025-12-31,2.4,0.00,0.00,0.00,0.30,2025-11-12 19:26:00
"New Zealand: Expected Annual CPI: 2 Years from Now (NSA, % Chg)",2.3,2025-12-31,2.3,0.00,0.00,0.00,0.20,2025-11-12 19:26:00
"New Zealand: Expected Annual CPI: 5 Years from Now (NSA, % Chg)",2.2,2025-12-31,2.3,-0.10,-0.97,0.00,0.00,2025-11-12 19:26:00


In [25]:
rbnz_sentiment_collection.dashboard.plots(show_0=True)

In [26]:
Markdown(f"_Notebook updated at {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')}_")

_Notebook updated at 2025-11-12 19:26_